In [4]:
import requests
from pathlib import Path
from datetime import date

# --- paste the URL you copied in step-1 ---
BASE_URL_POST = (
    "https://arctic-shift.photon-reddit.com/api/posts/search?"
)

# https://arctic-shift.photon-reddit.com/api/posts/search?subreddit=fednews&before=1743465600000&limit=auto&sort=asc&after=1735689600000&meta-app=download-tool
1735689600
# Build your query parameters; change as needed
params = {
    "subreddit": "fednews",
    # convert the date to utc timestamp
    # 2025-01-01 00:00:00 UTC = 1735689600
    # 2025-01-31 23:59:59 UTC = 1743465599
    "before": "1743465599", # data fetch ending date
    "limit" : "auto",
    "sort" : "asc",
    "after":  "1735689600", # data fetch starting date
    "meta-app" : "download-tool"
}

# Where to put the data
out_path = Path(f"{params['subreddit']}_posts.jsonl")

CHUNK = 1 << 14          # 16 kB blocks

print(f"➜ Downloading to {out_path.resolve()}")
with requests.get(BASE_URL_POST, params=params, stream=True, timeout=30) as r:
    r.raise_for_status()                     # 4xx/5xx → exception
    with out_path.open("wb") as f:
        for chunk in r.iter_content(CHUNK):
            if chunk:                       # skip keep-alive chunks
                f.write(chunk)

print("✅ Done!")


➜ Downloading to /home/csgrads/shahr072/sc_proj/data_collection/fednews_posts.jsonl


✅ Done!


In [7]:
import json
from datetime import datetime

max_created_utc = float('-inf')
min_created_utc = float('inf')

with open('fednews_comments_2025-01-01_to_2025-05-01_filtered.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        created_utc = data.get('created_utc', 0)
        if created_utc > max_created_utc:
            max_created_utc = created_utc
        if created_utc < min_created_utc:
            min_created_utc = created_utc

print("Max created_utc:", max_created_utc, "=>", datetime.utcfromtimestamp(max_created_utc))
print("Min created_utc:", min_created_utc, "=>", datetime.utcfromtimestamp(min_created_utc))


Max created_utc: 1746075561 => 2025-05-01 04:59:21
Min created_utc: 1735711244 => 2025-01-01 06:00:44


/tmp/ipykernel_2088608/4220833216.py:16: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print("Max created_utc:", max_created_utc, "=>", datetime.utcfromtimestamp(max_created_utc))
/tmp/ipykernel_2088608/4220833216.py:17: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print("Min created_utc:", min_created_utc, "=>", datetime.utcfromtimestamp(min_created_utc))


In [2]:
count = 0
with open('layoffs_comments_2022-01-01_to_2025-05-01_filtered.jsonl', 'r') as file:
    for _ in file:
        count += 1

print("Total number of comments:", count)


Total number of comments: 418769
